In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
from cycler import cycler
import re

# Auxiliary functions used to generate plots

In [ ]:
def plot_comparison_metric(competitors: dict[str, pd.DataFrame],
                           reference_competitor : str,
                           column : str,
                           title_plot : str,
                           y_label : str,
                           x_limit : tuple[float, float],
                           fig_size : tuple[float, float],
                           markevery : int,
                           save_path : str) :
    ''' 
    This function generates a plot that compares various scoring plugin under some metric as the % of requested
    GPU cluster resources increases.
    '''

    # Set the font size for all plot elements
    plt.rcParams.update({'font.size': 14})
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=fig_size)
    line_styles = ['-', '--', ':', '-.'] * 2
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
    markers = ['o', 's', 'D', 'v', '^', '<', '>', 'x']  # markers for linespoints
    ax1.set_prop_cycle(cycler('linestyle', line_styles) + cycler('color', colors) + cycler('marker', markers))
    
    for i, (k, v) in enumerate(competitors.items()) :
        if k == reference_competitor: continue
        ax1.plot(v.index, v[column], label=k, markevery=(i % markevery, markevery))
    ax1.plot(competitors[reference_competitor].index,
             competitors[reference_competitor][column],
             label=reference_competitor,
             markevery=(len(competitors) % markevery, markevery))

    ax1.set_xlim(x_limit)
    ax1.set_xlabel('Fraction of datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(y_label)
    ax1.legend(fontsize='small', ncol=2)
    plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path, format='pdf')
    
    plt.show()

In [ ]:
def plot_energy_savings(competitors_pwr: dict[str, pd.DataFrame], 
                        reference_competitor : str, 
                        column_power : str, 
                        title_plot : str,
                        markevery : int,
                        fig_size : tuple[float, float],
                        save_path : str) :

    # Set the font size for all plot elements
    plt.rcParams.update({'font.size': 14})
    
    fig, ax1 = plt.subplots(figsize=fig_size)
    line_styles = ['-', '--', ':', '-.'] * 2
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
    markers = ['o', 's', 'D', 'v', '^', '<', '>', 'x']  # markers for linespoints
    ax1.set_prop_cycle(cycler('linestyle', line_styles) + cycler('color', colors) + cycler('marker', markers))
    
    reference = competitors_pwr[reference_competitor]
    for i, (k, v) in enumerate(competitors_pwr.items()) :
        if k == reference_competitor: continue
        ax1.plot(v.index, (reference[column_power] - v[column_power]) / reference[column_power] * 100, label=k, markevery=(i % markevery, markevery))

    ax1.set_xlim((0, 1))
    ax1.set_xlabel('Fraction of datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(f'Percentage power savings vs {reference_competitor}')
    ax1.legend(fontsize='small', ncol=2)
    plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path, format='pdf')
    
    plt.show()

In [ ]:
def plot_stacked(dataframe : pd.DataFrame,
                 columns : list[str],
                 y_label : str,
                 save_path : str) :
    ''' %
    This function generates a plot showing the CPU and GPU energy consumption of a cluster under a certain scheduling policy.
    '''

    # Set the font size for all plot elements
    plt.rcParams.update({'font.size': 14})
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(8, 4))

    ax1.stackplot(dataframe.index, dataframe['power_cluster_CPU'], dataframe['power_cluster_GPU'], labels=['CPU', 'GPU'])
    ax1.set_xlabel('Fraction of datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(y_label)
    ax1.legend(fontsize='small', loc='upper left', ncol=2)
    ax1.set_xlim((0,1))
    plt.grid(True)

    # Creating a secondary y-axis for the ratio
    ax2 = ax1.twinx()
    ax2.plot(dataframe.index, dataframe['power_cluster_GPU'] / (dataframe['power_cluster_CPU'] + dataframe['power_cluster_GPU']), 
             color='black', linestyle='-.',
             label='Frac. power used by GPUs')
    ax2.set_ylabel('Fraction of power used by GPUs')
    ax2.legend(fontsize='small', loc='upper right', ncol=2)
    plt.tight_layout()

    plt.savefig(save_path, format='pdf')
    plt.show()

### Read the parsed results

In [ ]:
dict_pwr_final_res = None
dict_frag_final_res = None
dict_sched_final_res = None
dict_efficiency = None

with open('dict_pwr_final_res.pkl', 'rb') as f:
    dict_pwr_final_res = pickle.load(f)

with open('dict_frag_final_res.pkl', 'rb') as f:
    dict_frag_final_res = pickle.load(f)

with open('dict_sched_final_res.pkl', 'rb') as f:
    dict_sched_final_res = pickle.load(f)

with open('dict_efficiency.pkl', 'rb') as f:
    dict_efficiency = pickle.load(f)

In [ ]:
dict_pwr_final_res['openb_pod_list_default'].keys()

# Generation of plots

In [ ]:
# Generate the directory of the plots, if needed.
dir_plots = './energy_aware_plots/'
if not os.path.exists(dir_plots): os.makedirs(dir_plots)

### Plot concerning power comsumption of CPUs and GPUs with FGD when considering the Default trace.

In [ ]:
# dict_pwr_final_res['openb_pod_list_default']['FGD']
plot_stacked(dict_pwr_final_res['openb_pod_list_default']['FGD'],
             ['power_cluster_CPU', 'power_cluster_GPU'],
             'Power Consumption (W)',
             dir_plots + "pwrcons_default_FGD.pdf")

### Plots concerning power savings of linear combinations of PWR and FGS vs plain FGD

In [ ]:
reference_competitor = 'FGD'
level = 'openb_pod_list_default'

tmp_pwr_res = {k : v for k, v in dict_pwr_final_res[level].items() if ('PWR' in k) or ('FGD' in k)}

# Plot the energy savings achieved with some competitor w.r.t. the reference competitor.
# X-axis represents the arrived workloads in % of GPU resources available in the cluster. 
plot_energy_savings(tmp_pwr_res, reference_competitor, 
                    "power_cluster", 
                    None,
                    12,
                    (8, 5),
                    dir_plots + "pwrsaving_special_" + level + '.pdf')



tmp_eff = {k : v for k, v in dict_efficiency[level].items() if ('PWR' in k) or ('FGD' in k)}
# Plot the GPU resource allocation ratio achieved by the various competitors across the various batches of experiments.
plot_comparison_metric(tmp_eff,
                       reference_competitor,
                       'usage_efficiency',
                       None,
                       'GPU Resource Allocation Ratio',
                       (0.8,1),
                       (8, 5),
                       3,
                       dir_plots + "gpuocc_special_" + level + '.pdf')

In [ ]:
reference_competitor = 'FGD'
set_ignored_score_plugins = {'PWR', 'PWR_500_FGD_500', 'PWR_300_FGD_700', 'PWR_25_FGD_975'} # Set of plugins ignored
                                                                                            # for this set of plots.
for level in dict_pwr_final_res.keys() :

    print(f"Generating plots for {level}...")

    # Filter the policies to show in the plot.
    tmp_final_res = {k : v for k, v in dict_pwr_final_res[level].items() if k not in set_ignored_score_plugins}

    # Set up the title.
    title = None
    perc = re.search(r'\d{2,3}$', level)
    if perc is not None: perc = perc.group()
    if 'gpushare' in level: title = f"{perc}% of GPU resources requested by sharing-GPU tasks"
    if 'multigpu' in level: title = f"{perc}% increase in GPU resources requested by multi-GPU tasks"
    if 'gpuspec' in level: title = f"{perc}% of GPU-constrained tasks"
    
    # Plot the energy savings achieved with some competitor w.r.t. the reference competitor.
    # X-axis represents the arrived workloads in % of GPU resources available in the cluster. 
    plot_energy_savings(tmp_final_res, reference_competitor, 
                        "power_cluster", 
                        title,
                        20,
                        (8, 4),
                        dir_plots + "pwrsaving_" + level + '.pdf')

### Plots concerning the GPU resource allocation ratio metric

In [ ]:
policies = {'PWR_50_FGD_950', 'PWR_100_FGD_900', 'PWR_200_FGD_800', 'FGD', 'BestFit', 'PWREXP'}
filter_dict_eff = {}
for k in dict_efficiency.keys() :
    filter_dict_eff[k] = {k2 : v for k2, v in dict_efficiency[k].items() if k2 not in set_ignored_score_plugins} # Select only the policies in "policies".

In [ ]:
reference_competitor = 'FGD'
for level in filter_dict_eff.keys() :

    print(f"Generating plots for {level}...")

    ### Plot the GPU resource allocation ratio achieved by the various competitors across the various batches of experiments. ###

    # Set up the title.
    title = None
    perc = re.search(r'\d{2,3}$', level)
    if perc is not None: perc = perc.group()
    if 'gpushare' in level: title = f"{perc}% of GPU resources requested by sharing-GPU tasks"
    if 'multigpu' in level: title = f"{perc}% increase in GPU resources requested by multi-GPU tasks"
    if 'gpuspec' in level: title = f"{perc}% of GPU-constrained tasks"

    # Set the zoom in on the x-axis for the considered set of experiments.
    # Also, set the frequency of the markers, which is computed as a function of the interval considered on the X axis.
    x_inf = 0.
    if 'default' in level: x_inf = 0.9
    if ('gpushare' in level) : x_inf = 0.875 if int(perc) == 40 else 0.8
    if ('multigpu' in level): x_inf = 0.9
    markevery = int(20 * (1 - x_inf))

    # Generate the plot.
    plot_comparison_metric(filter_dict_eff[level],
                           reference_competitor,
                           'usage_efficiency',
                           title,
                           'GPU Resource Allocation Ratio',
                           (x_inf,1),
                           (8, 4),
                           markevery,
                           dir_plots + "gpuocc_" + level + '.pdf')